In [191]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.preprocessing import scale
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

train_salary=pd.read_csv("C:\\Users\\kp\\Pictures\\Assignments\\Support Vector Machines\\SalaryData_Train(1).csv",header=0)
test_salary=pd.read_csv("C:\\Users\\kp\\Pictures\\Assignments\\Support Vector Machines\\SalaryData_Test(1).csv",header=0)

def data_visualization(train_salary):
    
    sb.pairplot(data=train_salary)
    plt.show()
#Pair-Plot shows that the Numerical features present in the dataset are not highly correlated and hence all these features can be used 
#in our Model Building

    pd.crosstab(train_salary.Salary, train_salary.workclass).plot(kind="bar")
    plt.show()
#The plot clearly says that there is a Significant Variation in the Salary for the WorkClass Private rather than Other 
#Workclassess comparatively 
    
    return None

#data_visualization(train_salary)
train_salary

,age,workclass,education,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native,Salary
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30156,27,Private,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
30157,40,Private,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
30158,58,Private,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
30159,22,Private,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [192]:
#Data Preprocessing for both Train and Test Datasets

def data_preprocessing(dataframe):

    dataframe["Salary>50K"] = 0
    dataframe.loc[dataframe.Salary == ' >50K',"Salary>50K"] = 1
    dataframe = dataframe.reindex(columns=['Salary>50K','age', 'workclass', 'education', 'educationno', 'maritalstatus','occupation', 
                            'relationship', 'race', 'sex', 'capitalgain','capitalloss', 'hoursperweek', 'native','Salary'])

    dummies=pd.get_dummies(dataframe[['workclass','education','maritalstatus','occupation','relationship','race','sex','native']])

    dataframe.drop(['workclass','education','maritalstatus','occupation','relationship','race','sex','native','educationno','Salary'],axis=1,inplace=True)
    dataframe = pd.concat([dataframe,dummies],axis=1)


#Standardizing Each Column Values
    standard_dataframe = pd.DataFrame(data = scale(dataframe.iloc[:,1:]), columns = dataframe.columns[1:])
    standard_dataframe['Salary>50K']=dataframe['Salary>50K']
    
    return standard_dataframe

standard_train_salary=data_preprocessing(train_salary)
standard_test_salary=data_preprocessing(test_salary)
standard_train_salary

,age,capitalgain,capitalloss,hoursperweek,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Private,workclass_ Self-emp-inc,workclass_ Self-emp-not-inc,workclass_ State-gov,...,native_ Puerto-Rico,native_ Scotland,native_ South,native_ Taiwan,native_ Thailand,native_ Trinadad&Tobago,native_ United-States,native_ Vietnam,native_ Yugoslavia,Salary>50K
0,0.042779,0.146085,-0.218508,-0.077735,-0.179651,-0.271246,-1.682106,-0.192155,-0.300567,4.752021,...,-0.060225,-0.019101,-0.048576,-0.037343,-0.023748,-0.024437,0.310812,-0.046114,-0.023038,0
1,0.880261,-0.147447,-0.218508,-2.331495,-0.179651,-0.271246,-1.682106,-0.192155,3.327045,-0.210437,...,-0.060225,-0.019101,-0.048576,-0.037343,-0.023748,-0.024437,0.310812,-0.046114,-0.023038,0
2,-0.033356,-0.147447,-0.218508,-0.077735,-0.179651,-0.271246,0.594493,-0.192155,-0.300567,-0.210437,...,-0.060225,-0.019101,-0.048576,-0.037343,-0.023748,-0.024437,0.310812,-0.046114,-0.023038,0
3,1.108665,-0.147447,-0.218508,-0.077735,-0.179651,-0.271246,0.594493,-0.192155,-0.300567,-0.210437,...,-0.060225,-0.019101,-0.048576,-0.037343,-0.023748,-0.024437,0.310812,-0.046114,-0.023038,0
4,-0.794703,-0.147447,-0.218508,-0.077735,-0.179651,-0.271246,0.594493,-0.192155,-0.300567,-0.210437,...,-0.060225,-0.019101,-0.048576,-0.037343,-0.023748,-0.024437,-3.217378,-0.046114,-0.023038,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30156,-0.870838,-0.147447,-0.218508,-0.244681,-0.179651,-0.271246,0.594493,-0.192155,-0.300567,-0.210437,...,-0.060225,-0.019101,-0.048576,-0.037343,-0.023748,-0.024437,0.310812,-0.046114,-0.023038,0
30157,0.118914,-0.147447,-0.218508,-0.077735,-0.179651,-0.271246,0.594493,-0.192155,-0.300567,-0.210437,...,-0.060225,-0.019101,-0.048576,-0.037343,-0.023748,-0.024437,0.310812,-0.046114,-0.023038,1
30158,1.489339,-0.147447,-0.218508,-0.077735,-0.179651,-0.271246,0.594493,-0.192155,-0.300567,-0.210437,...,-0.060225,-0.019101,-0.048576,-0.037343,-0.023748,-0.024437,0.310812,-0.046114,-0.023038,0
30159,-1.251511,-0.147447,-0.218508,-1.747187,-0.179651,-0.271246,0.594493,-0.192155,-0.300567,-0.210437,...,-0.060225,-0.019101,-0.048576,-0.037343,-0.023748,-0.024437,0.310812,-0.046114,-0.023038,0


In [193]:
features=standard_train_salary.columns[0:101]
target=standard_train_salary.columns[101]
features

Index(['age', 'capitalgain', 'capitalloss', 'hoursperweek',
       'workclass_ Federal-gov', 'workclass_ Local-gov', 'workclass_ Private',
       'workclass_ Self-emp-inc', 'workclass_ Self-emp-not-inc',
       'workclass_ State-gov',
       ...
       'native_ Portugal', 'native_ Puerto-Rico', 'native_ Scotland',
       'native_ South', 'native_ Taiwan', 'native_ Thailand',
       'native_ Trinadad&Tobago', 'native_ United-States', 'native_ Vietnam',
       'native_ Yugoslavia'],
      dtype='object', length=101)

In [194]:
#Model Building

model_svm=SVC(C=5.0,kernel = "rbf",gamma='auto',class_weight='balanced',random_state=8)
model_svm.fit(standard_train_salary[features],standard_train_salary[target])
train_preds = model_svm.predict(standard_train_salary[features])
test_preds= model_svm.predict(standard_test_salary[features])

# confusion matrix using pandas CrossTab
print("Confusion Matrix: \n",confusion_matrix(standard_train_salary[target],train_preds))

Confusion Matrix: 
 [[18379  4274]
 [  733  6775]]


In [195]:
#Classification Report on Model's Performance for individual Classes
print("Classification Report of SVM Model: \n",classification_report(standard_train_salary[target],train_preds))

Classification Report of SVM Model: 
               precision    recall  f1-score   support

           0       0.96      0.81      0.88     22653
           1       0.61      0.90      0.73      7508

    accuracy                           0.83     30161
   macro avg       0.79      0.86      0.81     30161
weighted avg       0.87      0.83      0.84     30161



In [196]:
# Train Accuracy
print("Accuracy achieved on Training Dataset by Support Vector Machines (%): ",100*(np.mean(standard_train_salary[target]==train_preds)))

# Test Accuracy
print("Accuracy achieved on Test Dataset by Support Vector Machines (%): ",100*(np.mean(standard_test_salary[target]==test_preds)))

#Tried Different SVM's hyperparameter Settings but it didn't improve Accuracy rate due to imbalanced Dataset,this is the Optimal Accuracy Levels that i can get without Overfitting the data

Accuracy achieved on Training Dataset by Support Vector Machines (%):  83.39909154205762
Accuracy achieved on Test Dataset by Support Vector Machines (%):  79.8140770252324
